

#### NASDAQ, NYSE, AMEX, (stock only)

In [3]:
import pandas as pd
import numpy as np
import ssl
import certifi
from urllib.request import urlopen
import urllib.parse
import json
from tqdm import tqdm
from datetime import datetime, date
from pandas.tseries.offsets import MonthEnd

In [ ]:
def get_adata_annual(adata, data_name):
    data = adata.pivot_table(index='date', columns='ticker', values=data_name).astype('float')
    data = data.ffill(limit=11)  # Replace fillna(method='ffill') with ffill()
    data = data.loc[[x for x in data.index if x.month == 6]].reindex(data.index).ffill(limit=11).dropna(how='all')
    return data

def get_pivot_data(df, col_name):
    df_pivot = df.pivot(index ='date', columns ='ticker', values = col_name).astype('float')
    df_pivot.index = pd.to_datetime(df_pivot.index)
    return df_pivot

# 지난달 말일을 확인하고 end_date 변수에 입력하는 함수 정의
def get_last_month_end():
    return (pd.Timestamp.today() - MonthEnd(1)).strftime('%Y-%m-%d')

In [ ]:
path = '/content/drive/MyDrive/Stock_Investment_Strategy/Factor_Model/Data/US_stock_prc/ticker_list_2025-05-28.csv'
info = pd.read_csv(path)
info = info.drop(columns='Unnamed: 0')
# 유니크한 티커 리스트 생성
tic_list = info['ticker'].dropna().astype(str).unique().tolist()

# 필터링: 문자 수가 5개 이상이거나 '-'를 포함하는 티커 제외
filtered_tic_list = [tic for tic in tic_list if len(tic) < 5 and '-' not in tic]

In [ ]:
# end_date 변수 설정
end_date = get_last_month_end()
print("지난달 말일:", end_date)

지난달 말일: 2025-04-30


# 일간 데이터 (주가, 시가총액)


#### 일간 주가

In [7]:
def get_jsonparsed_data(url):
    # SSL 컨텍스트를 생성합니다.
    context = ssl.create_default_context(cafile=certifi.where())

    # SSL 컨텍스트를 사용하여 URL을 엽니다.
    response = urlopen(url, context=context)
    data = response.read().decode("utf-8")
    return json.loads(data)

# 기간을 조정하려면 시작 날짜와 종료 날짜를 지정합니다.
start_date = "2003-12-01"
end_date = end_date

prc_list = []
prc_error_list = []

# 여기서 tic_list가 이미 정의되어 있다고 가정합니다. tic_list가 정의되어 있지 않다면 정의해 주셔야 합니다.
# tic_list = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'BABA']  # 예시 ticker 리스트

for tick in tqdm(tic_list):

    try:
        # base_url에 날짜 범위와 API 키를 포함시킵니다.
        base_url = f"https://financialmodelingprep.com/api/v3/historical-price-full/{tick}"
        query_parameters = {
            "from": start_date,
            "to": end_date,
            "apikey": "hT0gAk87j9xZx4PlBApvBqfVL5IahvgV"
        }

        # URL 인코딩을 사용하여 전체 URL을 만듭니다.
        url = f"{base_url}?{urllib.parse.urlencode(query_parameters)}"
        json_data = get_jsonparsed_data(url)

        # JSON 데이터에서 필요한 부분을 추출하여 DataFrame으로 변환합니다.
        historical_data = json_data.get('historical', [])

        # DataFrame으로 변환
        df = pd.DataFrame(historical_data)
        df['ticker'] = tick
        prc_list.append(df)
    except Exception as e:
        print(f"Error fetching data for {tick}: {e}")
        prc_error_list.append(tick)

prc_df = pd.concat(prc_list)

today_date = datetime.now().strftime("%Y-%m-%d")


100%|██████████| 6126/6126 [1:48:41<00:00,  1.06s/it]


In [8]:
prc_df.to_csv(f'/content/drive/MyDrive/Stock_Investment_Strategy/Factor_Model/Data/US_stock_prc/daily_price_{today_date}.csv')

In [ ]:
# today_date = datetime.now().strftime("%Y-%m-%d")
# # prc_raw = pd.read_csv('/content/drive/MyDrive/Stock_Investment_Strategy/Factor_Model/Data/US_stock_prc/daily_price_{today_date}.csv')
# # prc_raw

#### 일간 주가, 월간 주가 산출 (단순 종가기준)

In [9]:
prc_raw = prc_df

prc_pivot = get_pivot_data(prc_raw, 'close')
prc_pivot.index = pd.to_datetime(prc_pivot.index)
prc_month_end = prc_pivot.resample('M').last()

adjprc_pivot = get_pivot_data(prc_raw, 'adjClose')
adjprc_pivot.index = pd.to_datetime(adjprc_pivot.index)
adjprc_month_end = adjprc_pivot.resample('M').last()

ret_monthly = adjprc_month_end.pct_change()
ret_daily = adjprc_pivot.pct_change()

prc_month_end.to_csv(fr'C:\Users\82108\OneDrive\바탕 화면\investment\investment_strategy/monthly_prc_{today_date}.csv')
ret_monthly.to_csv(fr'C:\Users\82108\OneDrive\바탕 화면\investment\investment_strategy/monthly_adjret_{today_date}.csv')
ret_daily.to_csv(fr'C:\Users\82108\OneDrive\바탕 화면\investment\investment_strategy/daily_adjret{today_date}.csv')

<ipython-input-9-f06af6f10678>:5: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  prc_month_end = prc_pivot.resample('M').last()
<ipython-input-9-f06af6f10678>:9: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  adjprc_month_end = adjprc_pivot.resample('M').last()
<ipython-input-9-f06af6f10678>:11: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  ret_monthly = adjprc_month_end.pct_change()
<ipython-input-9-f06af6f10678>:12: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  ret_daily = adjprc_p

### 가장 최근의 marketcap data  추출

In [12]:
# url = ("https://financialmodelingprep.com/api/v3/historical-market-capitalization/AAPL?&from=2004-10-10&to=2009-08-29&apikey=hT0gAk87j9xZx4vgV")
# raw = get_jsonparsed_data(url)

import urllib.parse
import requests
import pandas as pd
from tqdm import tqdm

# API 호출을 위한 함수 정의
def get_jsonparsed_data(url):
    response = requests.get(url)
    return response.json()

# 기간을 조정하려면 시작 날짜와 종료 날짜를 지정합니다.
start_date = "2024-08-30"
end_date = end_date
apikey = "hT0gAk87j9xZx4PlBApvBqfVL5IahvgV"

# tic_list = ["AAPL", "MSFT", "GOOGL"]  # 예시로 사용할 ticker 리스트

prc_list = []
prc_error_list = []

for tick in tqdm(filtered_tic_list):

    try:
        # URL을 구성합니다.
        base_url = f"https://financialmodelingprep.com/api/v3/historical-market-capitalization/{tick}"
        query_parameters = {
            "from": start_date,
            "to": end_date,
            "apikey": apikey
        }

        # URL 인코딩을 사용하여 전체 URL을 만듭니다.
        url = f"{base_url}?{urllib.parse.urlencode(query_parameters)}"
        json_data = get_jsonparsed_data(url)

        # json_data가 리스트 형태로 반환되는 경우를 처리합니다.
        if isinstance(json_data, list):
            df = pd.DataFrame(json_data)
            df['ticker'] = tick
            prc_list.append(df)
        else:
            prc_error_list.append(tick)
            print(f"No data or unexpected format for {tick}")

    except Exception as e:
        prc_error_list.append(tick)
        print(f"Error processing {tick}: {e}")

# 결과를 하나의 DataFrame으로 병합합니다.
if prc_list:
    final_df = pd.concat(prc_list, ignore_index=True)
    print(final_df)
else:
    print("No data was retrieved.")

100%|██████████| 5419/5419 [34:14<00:00,  2.64it/s]


       symbol        date     marketCap ticker
0        ACVA  2025-04-30  2.445660e+09   ACVA
1        ACVA  2025-04-29  2.492276e+09   ACVA
2        ACVA  2025-04-28  2.455650e+09   ACVA
3        ACVA  2025-04-25  2.463974e+09   ACVA
4        ACVA  2025-04-24  2.470633e+09   ACVA
...       ...         ...           ...    ...
704697   ENOB  2024-09-06  6.322713e+07   ENOB
704698   ENOB  2024-09-05  5.818485e+07   ENOB
704699   ENOB  2024-09-04  5.286754e+07   ENOB
704700   ENOB  2024-09-03  5.327500e+07   ENOB
704701   ENOB  2024-08-30  6.097593e+07   ENOB

[704702 rows x 4 columns]


In [1]:
# 날짜를 기준으로 정렬
mkc_added = final_df.sort_values(by=['ticker', 'date'])

# 날짜 데이터를 datetime 형식으로 변환
mkc_added['date'] = pd.to_datetime(mkc_added['date'], errors='coerce')

# 날짜를 기준으로 정렬
mkc_added = mkc_added.sort_values(by=['ticker', 'date'])

# 월말 데이터 필터링
mkc_added['month_end'] = mkc_added['date'].dt.to_period('M')
month_end_df = mkc_added.loc[mkc_added.groupby(['ticker', 'month_end'])['date'].idxmax()]

# 'month_end' 컬럼 제거
market_cap_additional = month_end_df.drop(columns=['month_end'])

market_cap_additional = market_cap_additional[['date',	'marketCap',	'ticker']]
market_cap_additional = market_cap_additional.rename(columns={'date': 'edate'})

NameError: name 'final_df' is not defined

In [14]:
market_cap_additional_re = market_cap_additional[market_cap_additional['edate'] != '2024-08-30']
market_cap_additional_re['edate'] = market_cap_additional_re['edate'] + pd.offsets.MonthEnd(0)
market_cap_additional_re

<ipython-input-14-81f32c4aed9e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  market_cap_additional_re['edate'] = market_cap_additional_re['edate'] + pd.offsets.MonthEnd(0)


,edate,marketCap,ticker
163208,2024-09-30,4.305920e+10,A
163185,2024-10-31,3.778990e+10,A
163165,2024-11-30,3.945942e+10,A
163144,2024-12-31,3.842124e+10,A
163124,2025-01-31,4.333472e+10,A
...,...,...,...
289556,2024-12-31,2.553988e+08,ZYXI
289536,2025-01-31,2.499784e+08,ZYXI
289517,2025-02-28,2.298908e+08,ZYXI
289496,2025-03-31,7.014700e+07,ZYXI


### 기존 시가총액 데이터와 결합

In [15]:
market_cap_raw = pd.read_csv(fr'C:\Users\82108\OneDrive\바탕 화면\investment\investment_strategy\DATA\market_cap_all.csv')
market_cap_raw = market_cap_raw.drop(columns=['Unnamed: 0'])
market_cap_monthly = market_cap_raw.drop_duplicates(subset=['edate',  'ticker'], keep='first')

### 추가된 데이터와 기존 수집데이터와 결합 및 저장

In [16]:
marketcap__data = pd.concat([market_cap_monthly, market_cap_additional_re])
marketcap__data.to_csv(fr'C:\Users\82108\OneDrive\바탕 화면\investment\investment_strategy\DATA\market_cap_{today_date}.csv')

In [17]:
marketcap__data[marketcap__data['ticker'] == 'A'].tail(24)

,edate,ticker,marketCap
10068076,2019-08-31,A,2.197299e+10
10068098,2019-07-31,A,2.144769e+10
10068120,2019-06-30,A,2.329704e+10
10068140,2019-05-31,A,2.091960e+10
10068162,2019-04-30,A,2.449200e+10
10068183,2019-03-31,A,2.548046e+10
10068204,2019-02-28,A,2.518248e+10
10068223,2019-01-31,A,2.410785e+10
14596092,2024-08-31,A,4.142940e+10
14596114,2024-07-31,A,4.100600e+10


In [ ]:
for tick in tqdm(prc_error_list):

    try:
        # URL을 구성합니다.
        base_url = f"https://financialmodelingprep.com/api/v3/historical-market-capitalization/{tick}"
        query_parameters = {
            "from": start_date,
            "to": end_date,
            "apikey": apikey
        }

        # URL 인코딩을 사용하여 전체 URL을 만듭니다.
        url = f"{base_url}?{urllib.parse.urlencode(query_parameters)}"
        json_data = get_jsonparsed_data(url)

        # json_data가 리스트 형태로 반환되는 경우를 처리합니다.
        if isinstance(json_data, list):
            df = pd.DataFrame(json_data)
            df['ticker'] = tick
            prc_list.append(df)
        else:
            prc_error_list.append(tick)
            print(f"No data or unexpected format for {tick}")

    except Exception as e:
        prc_error_list.append(tick)
        print(f"Error processing {tick}: {e}")

# 결과를 하나의 DataFrame으로 병합합니다.
if prc_list:
    final_df2 = pd.concat(prc_list, ignore_index=True)
    print(final_df2)
else:
    print("No data was retrieved.")

 64%|██████▍   | 575/898 [01:33<00:45,  7.12it/s]

No data or unexpected format for XRTX
No data or unexpected format for PETZ


 64%|██████▍   | 577/898 [01:33<00:41,  7.66it/s]

No data or unexpected format for VNDA
No data or unexpected format for GP


 64%|██████▍   | 579/898 [01:33<00:45,  6.99it/s]

...

No data or unexpected format for HOFV
No data or unexpected format for STKH


1011it [02:41,  7.25it/s]

No data or unexpected format for CSCI
No data or unexpected format for ITRG


1036it [02:47,  6.17it/s]


       symbol        date    marketCap ticker
0        LANV  2025-02-28  232293920.0   LANV
1        LANV  2025-02-27  221735106.0   LANV
2        LANV  2025-02-26  224081509.0   LANV
3        LANV  2025-02-25  214695896.0   LANV
4        LANV  2025-02-24  222908307.0   LANV
...       ...         ...          ...    ...
539898   ITRG  2024-09-06   79604254.0   ITRG
539899   ITRG  2024-09-05   83381453.0   ITRG
539900   ITRG  2024-09-04   81364588.0   ITRG
539901   ITRG  2024-09-03   80497690.0   ITRG
539902   ITRG  2024-08-30   84690646.0   ITRG

[539903 rows x 4 columns]


In [ ]:
final_df2.to_csv('/content/drive/MyDrive/Stock_Investment_Strategy/Factor_Model/Data/US_stock_prc/market_cap_20250228.csv')

In [ ]:
final_df

,symbol,date,marketCap,ticker
0,LANV,2025-02-28,232293920.0,LANV
1,LANV,2025-02-27,221735106.0,LANV
2,LANV,2025-02-26,224081509.0,LANV
3,LANV,2025-02-25,214695896.0,LANV
4,LANV,2025-02-24,222908307.0,LANV
...,...,...,...,...
457976,LLL,2024-09-06,10667032.0,LLL
457977,LLL,2024-09-05,11714103.0,LLL
457978,LLL,2024-09-04,11844987.0,LLL
457979,LLL,2024-09-03,11452336.0,LLL


#### 최근 시가총액 데이터를 호출하고 결합하여 다시 저장

In [ ]:
tic_list = prc_raw['ticker'].unique().tolist()

In [ ]:
# url = ("https://financialmodelingprep.com/api/v3/historical-market-capitalization/AAPL?&from=2004-10-10&to=2009-08-29&apikey=hT0gAk87j9xZx4vgV")
# raw = get_jsonparsed_data(url)


import urllib.parse
import requests
import pandas as pd
from tqdm import tqdm

# API 호출을 위한 함수 정의
def get_jsonparsed_data(url):
    response = requests.get(url)
    return response.json()

# 기간을 조정하려면 시작 날짜와 종료 날짜를 지정합니다.
start_date = "2009-01-01"
end_date = "2013-12-31"
apikey = "hT0gAk87j9xZx4PlBApvBqfVL5IahvgV"

# tic_list = ["AAPL", "MSFT", "GOOGL"]  # 예시로 사용할 ticker 리스트

prc_list = []
prc_error_list = []

for tick in tqdm(tic_list):

    try:
        # URL을 구성합니다.
        base_url = f"https://financialmodelingprep.com/api/v3/historical-market-capitalization/{tick}"
        query_parameters = {
            "from": start_date,
            "to": end_date,
            "apikey": apikey
        }

        # URL 인코딩을 사용하여 전체 URL을 만듭니다.
        url = f"{base_url}?{urllib.parse.urlencode(query_parameters)}"
        json_data = get_jsonparsed_data(url)

        # json_data가 리스트 형태로 반환되는 경우를 처리합니다.
        if isinstance(json_data, list):
            df = pd.DataFrame(json_data)
            df['ticker'] = tick
            prc_list.append(df)
        else:
            prc_error_list.append(tick)
            print(f"No data or unexpected format for {tick}")

    except Exception as e:
        prc_error_list.append(tick)
        print(f"Error processing {tick}: {e}")

# 결과를 하나의 DataFrame으로 병합합니다.
if prc_list:
    final_df = pd.concat(prc_list, ignore_index=True)
    print(final_df)
else:
    print("No data was retrieved.")

100%|██████████| 5808/5808 [36:38<00:00,  2.64it/s]


        ticker symbol        date     marketCap
0         ORCL   ORCL  2013-12-31  1.750395e+11
1         ORCL   ORCL  2013-12-30  1.738042e+11
2         ORCL   ORCL  2013-12-27  1.737585e+11
3         ORCL   ORCL  2013-12-26  1.724318e+11
4         ORCL   ORCL  2013-12-24  1.707390e+11
...        ...    ...         ...           ...
2850708  SHIPW  SHIPW  2009-01-08  1.106365e+08
2850709  SHIPW  SHIPW  2009-01-07  1.101948e+08
2850710  SHIPW  SHIPW  2009-01-06  1.159364e+08
2850711  SHIPW  SHIPW  2009-01-05  1.148323e+08
2850712  SHIPW  SHIPW  2009-01-02  1.181447e+08

[2850713 rows x 4 columns]


In [ ]:
final_df.to_csv('/content/drive/MyDrive/Stock_Investment_Strategy/Factor_Model/Data/US_stock_prc/market_cap_2013.csv')

In [ ]:
import urllib.parse
import requests
import pandas as pd
from tqdm import tqdm


# API 호출을 위한 함수 정의
def get_jsonparsed_data(url):
    response = requests.get(url)
    return response.json()


# 기간을 조정하려면 시작 날짜와 종료 날짜를 지정합니다.
start_date = "2014-01-01"
end_date = "2018-12-31"
apikey = "hT0gAk87j9xZx4PlBApvBqfVL5IahvgV"

# tic_list = ["AAPL", "MSFT", "GOOGL"]  # 예시로 사용할 ticker 리스트

prc_list = []
prc_error_list = []

for tick in tqdm(tic_list):

    try:
        # URL을 구성합니다.
        base_url = f"https://financialmodelingprep.com/api/v3/historical-market-capitalization/{tick}"
        query_parameters = {
            "from": start_date,
            "to": end_date,
            "apikey": apikey
        }

        # URL 인코딩을 사용하여 전체 URL을 만듭니다.
        url = f"{base_url}?{urllib.parse.urlencode(query_parameters)}"
        json_data = get_jsonparsed_data(url)

        # json_data가 리스트 형태로 반환되는 경우를 처리합니다.
        if isinstance(json_data, list):
            df = pd.DataFrame(json_data)
            df['ticker'] = tick
            prc_list.append(df)
        else:
            prc_error_list.append(tick)
            print(f"No data or unexpected format for {tick}")

    except Exception as e:
        prc_error_list.append(tick)
        print(f"Error processing {tick}: {e}")

# 결과를 하나의 DataFrame으로 병합합니다.
if prc_list:
    final_df = pd.concat(prc_list, ignore_index=True)
    print(final_df)
else:
    print("No data was retrieved.")

100%|██████████| 5808/5808 [35:33<00:00,  2.72it/s]


        symbol        date     marketCap ticker
0         BABA  2018-12-31  3.530923e+11   BABA
1         BABA  2018-12-28  3.582958e+11   BABA
2         BABA  2018-12-27  3.566472e+11   BABA
3         BABA  2018-12-26  3.554880e+11   BABA
4         BABA  2018-12-24  3.397486e+11   BABA
...        ...         ...           ...    ...
3733756   UNVR  2015-06-23  2.776860e+09   UNVR
3733757   UNVR  2015-06-22  2.751149e+09   UNVR
3733758   UNVR  2015-06-19  2.802572e+09   UNVR
3733759   UNVR  2015-06-18  2.612306e+09   UNVR
3733760   UNVR  2015-06-17  2.262627e+09   UNVR

[3733761 rows x 4 columns]


In [ ]:
final_df.to_csv('/content/drive/MyDrive/Stock_Investment_Strategy/Factor_Model/Data/US_stock_prc/market_cap_2018.csv')


In [ ]:
# 기간을 조정하려면 시작 날짜와 종료 날짜를 지정합니다.
start_date = "2019-01-01"
end_date = "2023-12-31"
apikey = "hT0gAk87j9xZx4PlBApvBqfVL5IahvgV"

# tic_list = ["AAPL", "MSFT", "GOOGL"]  # 예시로 사용할 ticker 리스트

prc_list = []
prc_error_list = []

for tick in tqdm(tic_list):

    try:
        # URL을 구성합니다.
        base_url = f"https://financialmodelingprep.com/api/v3/historical-market-capitalization/{tick}"
        query_parameters = {
            "from": start_date,
            "to": end_date,
            "apikey": apikey
        }

        # URL 인코딩을 사용하여 전체 URL을 만듭니다.
        url = f"{base_url}?{urllib.parse.urlencode(query_parameters)}"
        json_data = get_jsonparsed_data(url)

        # json_data가 리스트 형태로 반환되는 경우를 처리합니다.
        if isinstance(json_data, list):
            df = pd.DataFrame(json_data)
            df['ticker'] = tick
            prc_list.append(df)
        else:
            prc_error_list.append(tick)
            print(f"No data or unexpected format for {tick}")

    except Exception as e:
        prc_error_list.append(tick)
        print(f"Error processing {tick}: {e}")

# 결과를 하나의 DataFrame으로 병합합니다.
if prc_list:
    final_df = pd.concat(prc_list, ignore_index=True)
    print(final_df)
else:
    print("No data was retrieved.")

final_df.to_csv('/content/drive/MyDrive/Stock_Investment_Strategy/Factor_Model/Data/US_stock_prc/market_cap_2023.csv')

100%|██████████| 5808/5808 [33:55<00:00,  2.85it/s]


        symbol        date     marketCap ticker
0         BABA  2023-12-29  1.951120e+11   BABA
1         BABA  2023-12-28  1.944324e+11   BABA
2         BABA  2023-12-27  1.915879e+11   BABA
3         BABA  2023-12-26  1.909334e+11   BABA
4         BABA  2023-12-22  1.894986e+11   BABA
...        ...         ...           ...    ...
5505237   UNVR  2019-01-08  2.931780e+09   UNVR
5505238   UNVR  2019-01-07  2.857180e+09   UNVR
5505239   UNVR  2019-01-04  2.758708e+09   UNVR
5505240   UNVR  2019-01-03  2.615476e+09   UNVR
5505241   UNVR  2019-01-02  2.655760e+09   UNVR

[5505242 rows x 4 columns]


In [ ]:
# 기간을 조정하려면 시작 날짜와 종료 날짜를 지정합니다.
start_date = "2024-01-01"
end_date = "2024-08-30"
apikey = "hT0gAk87j9xZx4PlBApvBqfVL5IahvgV"

# tic_list = ["AAPL", "MSFT", "GOOGL"]  # 예시로 사용할 ticker 리스트

prc_list = []
prc_error_list = []

for tick in tqdm(tic_list):

    try:
        # URL을 구성합니다.
        base_url = f"https://financialmodelingprep.com/api/v3/historical-market-capitalization/{tick}"
        query_parameters = {
            "from": start_date,
            "to": end_date,
            "apikey": apikey
        }

        # URL 인코딩을 사용하여 전체 URL을 만듭니다.
        url = f"{base_url}?{urllib.parse.urlencode(query_parameters)}"
        json_data = get_jsonparsed_data(url)

        # json_data가 리스트 형태로 반환되는 경우를 처리합니다.
        if isinstance(json_data, list):
            df = pd.DataFrame(json_data)
            df['ticker'] = tick
            prc_list.append(df)
        else:
            prc_error_list.append(tick)
            print(f"No data or unexpected format for {tick}")

    except Exception as e:
        prc_error_list.append(tick)
        print(f"Error processing {tick}: {e}")

# 결과를 하나의 DataFrame으로 병합합니다.
if prc_list:
    final_df = pd.concat(prc_list, ignore_index=True)
    print(final_df)
else:
    print("No data was retrieved.")

final_df.to_csv('/content/drive/MyDrive/Stock_Investment_Strategy/Factor_Model/Data/US_stock_prc/market_cap_202408.csv')

100%|██████████| 5808/5808 [32:10<00:00,  3.01it/s]


       symbol        date     marketCap ticker
0        BABA  2024-08-30  2.299231e+11   BABA
1        BABA  2024-08-29  2.236299e+11   BABA
2        BABA  2024-08-28  2.197656e+11   BABA
3        BABA  2024-08-27  2.248168e+11   BABA
4        BABA  2024-08-26  2.256724e+11   BABA
...       ...         ...           ...    ...
811098  SHIPW  2024-01-08  1.391793e+08  SHIPW
811099  SHIPW  2024-01-05  1.536494e+08  SHIPW
811100  SHIPW  2024-01-04  1.515550e+08  SHIPW
811101  SHIPW  2024-01-03  1.475567e+08  SHIPW
811102  SHIPW  2024-01-02  1.443200e+08  SHIPW

[811103 rows x 4 columns]


In [ ]:
### 데이터 통합 결합 후 다시 저장

mk_df1 = pd.read_csv('/content/drive/MyDrive/Stock_Investment_Strategy/Factor_Model/Data/US_stock_prc/market_cap_2008.csv')
mk_df2 = pd.read_csv('/content/drive/MyDrive/Stock_Investment_Strategy/Factor_Model/Data/US_stock_prc/market_cap_2013.csv')
mk_df3 = pd.read_csv('/content/drive/MyDrive/Stock_Investment_Strategy/Factor_Model/Data/US_stock_prc/market_cap_2018.csv')
mk_df4 = pd.read_csv('/content/drive/MyDrive/Stock_Investment_Strategy/Factor_Model/Data/US_stock_prc/market_cap_2023.csv')
mk_df5 = pd.read_csv('/content/drive/MyDrive/Stock_Investment_Strategy/Factor_Model/Data/US_stock_prc/market_cap_202408.csv')

In [ ]:
concated_mkt_data = pd.concat([mk_df1, mk_df2, mk_df3, mk_df4, mk_df5])

In [ ]:
concated_mkt_data['date'] =  pd.to_datetime(concated_mkt_data['date'])
concated_mkt_data['edate'] = concated_mkt_data['date'] + pd.offsets.MonthEnd(0)
mkt_data_us = concated_mkt_data[['edate', 'ticker', 'marketCap']]
mkt_data_us.to_csv('/content/drive/MyDrive/Stock_Investment_Strategy/Factor_Model/Data/US_stock_prc/market_cap_all.csv')

In [ ]:

mkt_data_us

,edate,ticker,marketCap
0,2008-12-31,ORCL,8.873865e+10
1,2008-12-31,ORCL,8.923915e+10
2,2008-12-31,ORCL,8.618610e+10
3,2008-12-31,ORCL,8.723715e+10
4,2008-12-31,ORCL,8.668660e+10
...,...,...,...
811098,2024-01-31,SHIPW,1.391793e+08
811099,2024-01-31,SHIPW,1.536494e+08
811100,2024-01-31,SHIPW,1.515550e+08
811101,2024-01-31,SHIPW,1.475567e+08


In [ ]:
len(pd.date_range('2004-01-31', '2024-08-31', freq='M'))

248

In [ ]:
# 기간을 조정하려면 시작 날짜와 종료 날짜를 지정합니다.
start_date = "2019-01-01"
end_date = "2023-12-31"
apikey = "hT0gAk87j9xZx4PlBApvBqfVL5IahvgV"

# tic_list = ["AAPL", "MSFT", "GOOGL"]  # 예시로 사용할 ticker 리스트

prc_list = []
prc_error_list = []

for tick in tqdm(tic_list):

    try:
        # URL을 구성합니다.
        base_url = f"https://financialmodelingprep.com/api/v3/historical-market-capitalization/{tick}"
        query_parameters = {
            "from": start_date,
            "to": end_date,
            "apikey": apikey
        }

        # URL 인코딩을 사용하여 전체 URL을 만듭니다.
        url = f"{base_url}?{urllib.parse.urlencode(query_parameters)}"
        json_data = get_jsonparsed_data(url)

        # json_data가 리스트 형태로 반환되는 경우를 처리합니다.
        if isinstance(json_data, list):
            df = pd.DataFrame(json_data)
            df['ticker'] = tick
            prc_list.append(df)
        else:
            prc_error_list.append(tick)
            print(f"No data or unexpected format for {tick}")

    except Exception as e:
        prc_error_list.append(tick)
        print(f"Error processing {tick}: {e}")

# 결과를 하나의 DataFrame으로 병합합니다.
if prc_list:
    final_df = pd.concat(prc_list, ignore_index=True)
    print(final_df)
else:
    print("No data was retrieved.")

final_df.to_csv('/content/drive/MyDrive/Stock_Investment_Strategy/Factor_Model/Data/US_stock_prc/market_cap_2023.csv')